# Colab setup

Preparazione dell'ambiente di esecuzione del colab notebook. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Colab_yelp/Yelp-Data-Analysis

In [ ]:
! git pull

In [ ]:
! pip install -q -U "tensorflow-text==2.8.*"
! pip install -q tf-models-official==2.7.0
! pip install keras-tuner

# TASK 1
Riconoscimento automatico del sentiment delle review di ristoranti pubblicate raccolte da Yelp.

Le reviews sono associate ad un voto in stelle, compreso tra 1 e 5. Per il progetto, le reviews con massimo tre stelle sono considerate negative me tre quelle da 4 e 5 stelle sono positive. 

In [ ]:
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Embedding
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from official.nlp import optimization

# from libraries import data_handler
from libraries.dataset import Dataset

import tensorflow.keras as keras
import tensorflow as tf
import keras_tuner as kt

import libraries.preprocessing_utils as prep_utils
import libraries.models_builders as models_builders
import libraries.filenames_generator as filenames  
import constants as const

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

print(tf.test.gpu_device_name())

## Data retrieving
Ottenimento dei dati relativi alle reviews, bilanciati sulla base del sentiment.
In questo specifico caso, sono richiesti 50_000 samples per ogni tipo di sentiment (per un totale di 100_000 samples).

L'oggetto review_data contiene tre field relativi ai subdataset da utilizzare nel progetto:
- train_data = (x_train, y_train)
- val_data = (x_val, y_val)
- test_data = (x_test, y_test)

Alla prima esecuzione, i tre diversi subset appena selezionati sono salvato sottoforma di csv, in questo modo, nelle seguenti esecuzioni, tali file saranno letti senza dover rieseguire il codice di lettura e splitting dei dati.  


In [ ]:
review_data = Dataset('review', 'sentiment')

#  50_000 elements for each class
review_data.split(['text'], 'sentiment', n_samples=50_000)

## Data preprocessing comune

Fase di preparazione dei raw data ottenuti precedentemente, al fine di ottenere testo pulito utilizzabile come base per i diversi modelli di machine learning da testare successivamente.

Le fasi comuni del preprocessing dei dati sono:
  - lowercasing
  - decontractions
  - rimozione delle stop-words
  - lemmatizzazione

Ogni modello aggiungerà ulteriori azioni di processamento dei dati, necessarie per adattare al meglio i dati al tipo di input atteso.


In [ ]:
prep_train_data = prep_utils.preprocess_texts(review_data.train_data[0]['text'], path= filenames.picked_cleaned_sentences(
        "train", "task1"))

prep_test_data = prep_utils.preprocess_texts(review_data.test_data[0]['text'], path= filenames.picked_cleaned_sentences(
        "test", "task1"))

prep_val_data = prep_utils.preprocess_texts(review_data.val_data[0]['text'], path= filenames.picked_cleaned_sentences(
        "val", "task1"))


## Training dei classificatori
I classificatori testati in questo studio sono:
- **Multinomial Naive Bayes**
- **LSTM**
- **BERT**

### Multinomial Naive Bayes

Il classificatore di tipo Multinomial Naive Bayes è stato addestrato su un subset ridotto di 30_000.

L'ulteriore preprocessing applicato ai dati è eseguito dall'oggetto CountVectorizer, il quale converte la collezione delle reviews in una matrice contenente il conteggio dei token.

In [ ]:
vectorizer = CountVectorizer()
nb_train_data = vectorizer.fit_transform(prep_train_data[:30_000]).toarray()

nb_model = MultinomialNB()
nb_model.fit(nb_train_data, review_data.train_data[1][:30_000])

Nella fase di testing del modello addestrato, l'accuratezza raggiunta è stata dell'86%.

In [ ]:
nb_test_data = vectorizer.transform(prep_test_data).toarray()
nb_model.score(nb_test_data, review_data.test_data[1])

## LSTM
Sono state testate diverse architetture delle RNN di tipo LSTM, al fine di trovare la combinazione di iperparametri che rendono le migliori performance.

Riguardo il preprocessing aggiuntivo sui dati, è stato utilizzato un tokenizer specifico, il quale restituisce per ogni review un vettore contenente gli indici delle parole nel dizionario (estratto dalla collezione di reviews).

Tali indici sono fondamentali per il recupero delle word embedding corrispondenti alle parole delle reviews, azione che avviene nel layer di tipo Embedding impostato come primo layer della rete neurale.


In [ ]:
tokenizer = prep_utils.get_tokenizer(review_data.train_data[0]['text'])

train_tokens = prep_utils.get_set_tokens(
    review_data.train_data[0]['text'], tokenizer, set='train', task='task1')

test_tokens = prep_utils.get_set_tokens(
    review_data.test_data[0]['text'], tokenizer, set='test', task='task1')

val_tokens = prep_utils.get_set_tokens(
    review_data.val_data[0]['text'], tokenizer, set='val', task='task1')

Esistono diverse metodologie per la definizione dei word vectors, dal training di reti come Word2Vec all'utilizzo di mapping già pre-trained

Nel caso di questo studio è stato scelto di creare un'embedding matrix a partire da un mapping già esistente, nello specifico quello messo a disposizione da Glove e addestrato su una grande mole di dati testuali estratti da twitter.
Sulla base di questo mapping, sono stati estratti, ed inseriti in una matrice, i vettori delle parole presenti nel dizionario e, se non presenti, queste sono stati rappresentati come 0-vector.

In [ ]:
e_matrix = prep_utils.get_embedding_matrix(const.word_embedding_filepath, 'task1',
                                            tokenizer, len(tokenizer.index_word)+1)

word_vector_dim = 100

vocab_size = len(tokenizer.word_index) +1
max_length = len(max(train_tokens, key=len))

embedding_layer = Embedding(vocab_size, word_vector_dim,
                            embeddings_initializer=Constant(e_matrix), trainable=False)

Inizio fase di tuning degli iperparametri, la ricerca è divisa in due trial diverse, caratterizzate principalmente da due batch size differenti. 

Gli iperparametri testati sono:
- numero di units (dimensione del vettore delle celle e hidden states)
- percentuale di dropout
- learning rate

Il training è gestito utilizzando la tecnica dell'early stopping, gestita dalla seguente callback, che termina il training dopo 15 epoche prive di miglioramenti.

In [ ]:
# define custom callbacks
stop_early_cb = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=15)

### Primo hypeperameters tuning trial

In [ ]:
project_name = "task1_lstm_adam_128"

builder = models_builders.get_rnn_builder(
    drop=[0.2, 0.5],
    units=[15, 20, 50, 80],
    lrate=[0.01, 0.001],
    optimizer=keras.optimizers.Adam,
    embedding_layer=embedding_layer)

tuner = kt.RandomSearch(
    builder,
    objective = 'val_accuracy',
    max_trials = 10,
    directory = const.tuner_path, project_name = project_name
)

tuner.search_space_summary()

In [ ]:
tuner.search(train_tokens, review_data.train_data[1],
             batch_size=128, epochs=1000,
             validation_data=(val_tokens, review_data.val_data[1]),
             callbacks=[
                 stop_early_cb,
                 tf.keras.callbacks.TensorBoard(const.logs_path + project_name, update_freq='epoch')],
             verbose=0)

#  executed

### Secondo hypeperameters tuning trial

In [ ]:
project_name = "task1_lstm_adam_64_new"

builder = models_builders.get_rnn_builder(
    drop=[0.2, 0.5],
    units=[100, 150],
    lrate=[0.1, 0.01, 0.001],
    optimizer=keras.optimizers.Adam,
    embedding_layer=embedding_layer)

tuner1 = kt.RandomSearch(
    builder,
    objective = 'val_accuracy',
    max_trials = 10,
    directory = const.tuner_path, project_name = "task1_lstm_adam_64"
)

tuner1.search_space_summary()

# executed
tuner1.search(train_tokens, review_data.train_data[1],
             batch_size=64, epochs=1000,
             validation_data=(val_tokens, review_data.val_data[1]),
             callbacks=[
                 stop_early_cb,
                 tf.keras.callbacks.TensorBoard(const.logs_path + project_name, update_freq='epoch')],
             verbose=0)

Il miglior modello trovato, in seguito ad entrambi i trials, fornisce un'accuratezza sul validation set del 90.2% e si presenta come una rete con le seguenti caratteristiche:
- dropout del 50%
- learning rate di 0.001
- 100 units

Tutte le statistiche di esecuzione sono visualizzabili su tensorboard.

## BERT
E' stata testata anche la tecnica BERT per questo task di classificazione, basandosi su un modello BERT pre-addestrato e procedendo con il suo fine-tuning.

Il modello scelto è stato scaricato da tensorflow-hub e consiste nella versione Small Bert, caratterizzara da un numero minore di transformer blocks.

In [ ]:
handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

Preprocessing aggiuntivo dei dati già processati, wrapping delle review nell'oggetto Dataset.

In [ ]:
#  prepariamo i dati
train_df = tf.data.Dataset.from_tensor_slices((prep_train_data, review_data.train_data[1]))
val_df = tf.data.Dataset.from_tensor_slices((prep_val_data, review_data.val_data[1]))

Creazione del modello.

In [ ]:
epochs = 5

steps_per_epoch = tf.data.experimental.cardinality(train_df).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps, # lr decay
                                          optimizer_type='adamw')
                                          

In [ ]:
model = models_builders.build_BERT_model()

model.compile(optimizer=optimizer,
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=tf.metrics.BinaryAccuracy())

Training del modello.

L'accuratezza raggiunta sul training set è del 91.11% e sul validation set del 90.41%. 

In [ ]:
batch = 16

history = model.fit(x=train_df.batch(batch),
                    validation_data=val_df.batch(batch),
                    epochs=epochs,
                    callbacks=[
                               tf.keras.callbacks.TensorBoard(const.logs_path + "bert_task1", update_freq='epoch')])

Il testing del modello su test set ha raggiunto un'accuratezza del 90.90%.

In [ ]:
test_df = tf.data.Dataset.from_tensor_slices((prep_test_data, review_data.test_data[1]))

loss, accuracy = model.evaluate(test_df.batch(1))